In [2]:
import pandas as pd
from datetime import datetime, timedelta
from openpyxl import Workbook
from openpyxl.styles import Font, Border, Side, Alignment, PatternFill
from openpyxl.utils.dataframe import dataframe_to_rows

In [3]:
# Загрузка данных
df_main = pd.read_excel('01.2025 Верный Торты и пирожные.xlsb')
df_sku = pd.read_excel('Справочник Кондитерка Фрэш.xlsx')
df_tt = pd.read_excel('Справочник ТТ.xlsx')

In [4]:
# 1. Расчет себестоимости
df_main['Себестоимость(руб.)'] = df_main['Цена вх.'] * df_main['Реализация шт.(кг)']

In [5]:
# Так как в исходной таблице ошибка в нумерации недель, вариант конвертации недель в месяцы не подходит (код конвертации ниже).
# В колонке "Месяц" принудительно установим значение 1                                                                         

# # 2. Конвертация недели в месяц
# def week_to_month(week_number, year=2025):
#     start_date = datetime(year, 1, 1)
#     week_date = start_date + timedelta(weeks=week_number-1)
#     return week_date.month

# df_main['Месяц'] = df_main['Неделя'].apply(week_to_month)
# df_main['Год'] = 2025

In [6]:
# 2. Принудительно устанавливаем месяц = 1 для всех записей
df_main['Месяц'] = 1
df_main['Год'] = 2025

In [7]:
# 3. Агрегация данных
numeric_cols = ['Реализация шт.(кг)', 'Реализация Сумма', 'Себестоимость(руб.)']
group_cols = ['Месяц', 'Год', '№ магазина', 'Код товара', 'Наименование товара']
df_month = df_main.groupby(group_cols)[numeric_cols].sum().reset_index()

In [8]:
# 4. Добавление глобальных кодов
df_month['Сеть'] = 'VERNYY'
df_month['Глобальный код ТТ'] = 'VERNYY' + df_month['№ магазина'].astype(str)
df_month['Глобальный Код SKU'] = 'VERNYY' + df_month['Код товара'].astype(str)

In [9]:
# 5. Получение наименований SKU из справочника
sku_mapping = df_sku.set_index('Глобальный Код SKU')['Наименование SKU'].to_dict()
df_month['Наименование SKU'] = df_month['Глобальный Код SKU'].map(sku_mapping)

In [10]:
# 6. Фильтрация и переименование столбцов
result_cols = [
    'Месяц', 'Год', 'Сеть', 'Глобальный код ТТ', 
    '№ магазина', 'Глобальный Код SKU', 'Наименование SKU',
    'Реализация шт.(кг)', 'Реализация Сумма', 'Себестоимость(руб.)'
]

In [11]:
final_df = df_month[result_cols].rename(columns={
    '№ магазина': 'Код ТТ',
    'Реализация шт.(кг)': 'Продажа(шт)',
    'Реализация Сумма': 'Оборот(руб.)'
})

In [12]:
# 7. Создание Excel файла с форматированием
wb = Workbook()
ws = wb.active
ws.title = "Отчет VERNYY"

In [13]:
# Заголовки
headers = [
    'Месяц', 'Год', 'Сеть', 'Глобальный код ТТ', 'Код ТТ',
    'Глобальный Код SKU', 'Наименование SKU', 
    'Продажа(шт)', 'Оборот(руб.)', 'Себестоимость(руб.)'
]

In [14]:
# Наполнение данными
for r in dataframe_to_rows(final_df, index=False, header=True):
    ws.append(r)

In [15]:
# Стили оформления
header_font = Font(bold=True, color="FFFFFF")
header_fill = PatternFill(start_color="4F81BD", end_color="4F81BD", fill_type="solid")
thin_border = Border(
    left=Side(style='thin'), 
    right=Side(style='thin'), 
    top=Side(style='thin'), 
    bottom=Side(style='thin')
)

In [16]:
# Применение стилей
for row in ws.iter_rows(min_row=1, max_row=1):
    for cell in row:
        cell.font = header_font
        cell.fill = header_fill
        cell.border = thin_border

for row in ws.iter_rows(min_row=2):
    for cell in row:
        cell.border = thin_border
        if cell.column_letter in ['G', 'H', 'I', 'J']:
            cell.number_format = '#,##0.00'
        elif cell.column_letter in ['A', 'B']:
            cell.number_format = '0'

In [17]:
# Автоподбор ширины столбцов
for col in ws.columns:
    max_length = 0
    column = col[0].column_letter
    for cell in col:
        cell.alignment = Alignment(horizontal='center', vertical='center')
        try:
            if len(str(cell.value)) > max_length:
                max_length = len(str(cell.value))
        except:
            pass
    adjusted_width = (max_length + 2) * 1.2
    ws.column_dimensions[column].width = adjusted_width

In [18]:
# Вывод первых 5 строк фрейма для проверки изменений
final_df.head(5)

,Месяц,Год,Сеть,Глобальный код ТТ,Код ТТ,Глобальный Код SKU,Наименование SKU,Продажа(шт),Оборот(руб.),Себестоимость(руб.)
0,1,2025,VERNYY,VERNYY1002,1002,VERNYY100593,Пирожное Чокопай Orion Сhoco Pie 360г,5,854.95,580.20
1,1,2025,VERNYY,VERNYY1002,1002,VERNYY100596,Пирожное Чокопай Orion Сhoco Pie 180г,17,1761.98,1035.98
2,1,2025,VERNYY,VERNYY1002,1002,VERNYY100920,Пирожное Медвежонок Барни шок.начинкой 150г,22,2184.78,2706.22
3,1,2025,VERNYY,VERNYY1002,1002,VERNYY100934,Пирожное Медвежонок Барни биск с нач в ассорт ...,13,1294.87,1599.13
4,1,2025,VERNYY,VERNYY1002,1002,VERNYY102454,Торт Черемушки Чародейка Классика 650г,22,9783.04,6512.88


In [19]:
# Фиксация заголовков
ws.freeze_panes = 'A2'

In [20]:
# Сохранение файла
wb.save("Отчет_VERNYY_2025.xlsx")
print("Файл успешно создан!")

Файл успешно создан!
